In [1]:
import pandas as pd
import re
from nltk import ngrams
from underthesea import word_tokenize
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import os
from sklearn.model_selection import GridSearchCV

/home/hoangtv/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
data = pd.read_csv('data_crawler.csv')

In [3]:
data

,Text,Sentiment
0,18/4 Lazada giao nhưng 19/4 tôi mới khui hộp m...,1
1,"Đã nhận được hàng, máy đẹp, giao hàng nhanh. C...",0
2,Tôi mua sản phẩm samsung Galaxy M10. Máy bị lỗ...,1
3,"Đặt Samsung Galaxy M10,giao hàng ngoài hộp M10...",1
4,"Hàng đẹp xài ngon so với giá tiền, nên mua cho...",0
...,...,...
117,"Shop bán hàng không có tâm, lừa đảo",1
118,"Shop không cho trả hàng, lừa dối khách hàng",1
119,"hàng tốt, mình sẽ mua thêm",0
120,"dkm, shop bán hàng cũ bào hàng mới, bố láo",1


In [4]:
def standardize_data(row):
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
 
    row = row.strip()
    return row

In [5]:
data['Text'] = data['Text'].apply(standardize_data)

In [6]:
def tokenizer(row):
    return word_tokenize(row, format="text")

In [7]:
data['Text'] = data['Text'].apply(tokenizer)

In [8]:
def embedding(X_train, X_test):
    global  emb
    # Xét các từ có tần suất xuất hiện trên 5 lần trong tập các văn bản và dưới 80% trong toàn bộ các văn bản
    emb = TfidfVectorizer(min_df=5, max_df=0.8,max_features=3000,sublinear_tf=True)
    emb.fit(X_train)
    X_train =  emb.transform(X_train)
    X_test = emb.transform(X_test)
 
    # Save pkl file
    joblib.dump(emb, 'tfidf.pkl')
    return X_train, X_test

In [9]:
X_train,X_test,y_train,y_test = train_test_split(data["Text"], data["Sentiment"],test_size=0.2)

In [10]:
X_train,X_test  = embedding(X_train, X_test)

In [11]:
model = svm.SVC(kernel='linear', C = 1)
model.fit(X_train,y_train)
joblib.dump(model, 'saved_model.pkl')

['saved_model.pkl']

In [12]:
model.score(X_test, y_test)

0.84